In [1]:
!wget "https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py"

--2024-07-29 07:05:00--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0.001s  

2024-07-29 07:05:01 (6.51 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
!wget 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'

--2024-07-29 07:25:53--  https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json
Resolving github.com (github.com)... 20.207.73.82
Connecting to github.com (github.com)|20.207.73.82|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexeygrigorev/llm-rag-workshop/main/notebooks/documents.json [following]
--2024-07-29 07:25:53--  https://raw.githubusercontent.com/alexeygrigorev/llm-rag-workshop/main/notebooks/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  2.63MB/s    in 0.2s    

2024-07-29 07:25:54 (2.63 MB/s) - ‘documents.json’ saved [658332

In [8]:
import json
import cohere
import os
from dotenv import load_dotenv
from minsearch import Index

load_dotenv()

documents = []

with open('documents.json', 'r') as file:

    docs = json.load(file)
    
    

In [9]:
for doc in docs:

    for document in doc['documents']:

        document['course'] = doc['course'] 
        documents.append(document)

In [10]:
textfields = ["text", "section", "question"]


indobject = Index(text_fields = textfields, keyword_fields = ['course'])
indobject.fit(documents)

In [11]:


def search(query, boost_dict = {"question": 3}, filter_dict =  {"course":"mlops-zoomcamp"} ):

    
    
    
    context = indobject.search(query = query, boost_dict = boost_dict, filter_dict = filter_dict
                    )
    
    return context

In [12]:
def build_prompt(query, related_docs):
    
    
    
    
    prompt = """
    You are an assistant for teaching online courses answer the question based on the context provided.
    Only the information available in the provided context should be used strictly adhere to this
    
    question:{query}
    
    context:{context}
    
    
    """.strip()
    
    
    context = ""
    for doc in related_docs:
        
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']} \n\n"
        
    
    prompt = prompt.format(query = query, context = context).strip()
    
    return prompt

In [13]:
def chat(prompt, model = "command-r-plus"):
    co_client = cohere.Client(api_key = os.environ["KEY"])
    
    response = co_client.chat( model = model, message = prompt)
    
    return response.text
    
    

In [17]:
query = "How to run spark engine ?"

In [14]:
context = search(query = query, filter_dict = {"course": "data-engineering-zoomcamp"})

In [15]:
context

[{'text': 'Change the working directory to the spark directory:\nif you have setup up your SPARK_HOME variable, use the following;\ncd %SPARK_HOME%\nif not, use the following;\ncd <path to spark installation>\nCreating a Local Spark Cluster\nTo start Spark Master:\nbin\\spark-class org.apache.spark.deploy.master.Master --host localhost\nStarting up a cluster:\nbin\\spark-class org.apache.spark.deploy.worker.Worker spark://localhost:7077 --host localhost',
  'section': 'Module 5: pyspark',
  'question': 'How to spark standalone cluster is run on windows OS',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'The docker will keep on crashing continuously\nNot working after restart\ndocker engine stopped\nAnd failed to fetch extensions pop ups will on screen non-stop\nSolution :\nTry checking if latest version of docker is installed / Try updating the docker\nIf Problem still persist then final solution is to reinstall docker\n(Just have to fetch images again else no issues)',
  'sectio

In [18]:
prompt = build_prompt(query = query, related_docs = context )

In [19]:
answer = chat(prompt = prompt)

In [20]:
answer

"To run a Spark standalone cluster on Windows OS, follow these steps: \n\n1. Change the working directory to the Spark directory:\n```\ncd %SPARK_HOME%\n```\nIf you haven't set up the `SPARK_HOME` variable, use the path to your Spark installation instead. \n\n2. Start the Spark Master by navigating to the bin folder:\n```\nbin\\spark-class org.apache.spark.deploy.master.Master --host localhost\n```\n\n3. Start the worker nodes by providing the Spark Master URL: \n```\nbin\\spark-class org.apache.spark.deploy.worker.Worker spark://localhost:7077 --host localhost\n```\n\nMake sure to replace `localhost:7077` with the appropriate IP address and port if your setup requires a different configuration. \n\nThese steps will help you run the Spark engine in standalone mode on a Windows operating system."